# ⚠️ Important Notice

This notebook (and repository) is deprecated.

For the latest python examples, please refer to the `llama-cloud-services` repository examples: 
https://github.com/run-llama/llama_cloud_services/tree/main/examples

---

# Getting Started: Building Agents over LlamaCloud RAG Pipelines

<a href="https://colab.research.google.com/github/run-llama/llamacloud-demo/blob/main/examples/getting_started_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook we show you how to build a function calling agent (powered by OpenAI) over RAG pipelines built with LlamaCloud.

Adding an agentic layer to RAG allows you to build in a layer of query planning and state management that allows you to ask multi-part complex questions over existing RAG pipelines and get back answers in a conversational manner.


## Initial Setup 

Let's start by importing some simple building blocks.  

The main thing we need is:
1. the OpenAI API (using our own `llama_index` LLM class)
2. a place to keep conversation history 
3. a definition for tools that our agent can use.

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.


In [ ]:
%pip install -U llama-index-indices-managed-llama-cloud
%pip install -U llama-index
%pip install -U llama-index-core

In [39]:
import nest_asyncio
nest_asyncio.apply()

from IPython.display import Markdown, display

Make sure your OPENAI_API_KEY is set. Otherwise explicitly specify the `api_key` parameter.

In [40]:
from llama_index.llms.openai import OpenAI
import os

os.environ["OPENAI_API_KEY"] = "<openai_api_key>"

llm = OpenAI(model="gpt-4.1")

## Build Two LlamaCloud Indexes

Our data sources are the 2021 Lyft and Uber 10K documents.

In contrast to the other getting started examples, in this notebook we will build **two** RAG pipelines: one for Uber and one for Lyft. This is for the sake of example; we can plug in both RAG pipelines as tools for the agent to reason over.

To create each index, follow the instructions:
1. You can download them here ([Uber 10K](https://www.dropbox.com/s/te0a2w227v27iag/uber_2021.pdf?dl=1), [Lyft 10K](https://www.dropbox.com/s/qctkz6nxhm0y5qe/lyft_2021.pdf?dl=1))
2. Follow instructions on `https://cloud.llamaindex.ai/` to signup for an account. Create a pipeline by uploading these documents.

In [ ]:
from llama_index.indices.managed.llama_cloud import LlamaCloudIndex

index_uber = LlamaCloudIndex(
  name="<index_uber>", 
  api_key="<api_key>"
)
index_lyft = LlamaCloudIndex(
  name="<index_lyft>", 
  api_key="<api_key>"
)

For each index, get a query engine from the index, which gives us an out-of-the-box RAG pipeline.

In [42]:
query_engine_uber = index_uber.as_query_engine(llm=llm)
query_engine_lyft = index_lyft.as_query_engine(llm=llm)

## OpenAI Agent over LlamaCloud RAG Pipelines

We convert both query engines to tools and pass it to a function calling OpenAI agent.

In [43]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

tool_uber = QueryEngineTool(
    query_engine=query_engine_uber,
    metadata=ToolMetadata(
        name="uber_10k",
        description=(
            "Provides information about Uber financials for year 2021. "
            "Use a detailed plain text question as input to the tool."
        ),
    ),
)
tool_lyft = QueryEngineTool(
    query_engine=query_engine_lyft,
    metadata=ToolMetadata(
        name="lyft_10k",
        description=(
            "Provides information about Lyft financials for year 2021. "
            "Use a detailed plain text question as input to the tool."
        ),
    ),
)

In [44]:
from llama_index.core.agent.workflow import FunctionAgent

agent = FunctionAgent(
    tools=[tool_uber, tool_lyft],
    llm=llm,
    system_prompt="You are a helpful assistant that can answer questions about Uber and Lyft.",
    verbose=True,
)

In [45]:
response = await agent.run(user_msg="Tell me both the tailwinds for Uber and Lyft?")

Running step init_run
Step init_run produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced no event
Running step call_tool
Running step call_tool
Step call_tool produced event ToolCallResult
Running step aggregate_tool_results
Step aggregate_tool_results produced no event
Step call_tool produced event ToolCallResult
Running step aggregate_tool_results
Step aggregate_tool_results produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced event StopEvent


In [46]:
display(Markdown(f"{response}"))

Here are the main tailwinds for both Uber and Lyft in 2021:

Uber:
- Significant increase in Gross Bookings (up 56% year-over-year), driven by strong growth in both Delivery and Mobility segments.
- Delivery segment saw substantial growth due to increased food delivery orders, higher basket sizes (influenced by stay-at-home demand from COVID-19), and expansion in U.S. and international markets.
- Mobility segment rebounded as trip volumes recovered from pandemic lows.
- Revenue increased by 57% year-over-year, with additional contributions from the Freight segment (especially after acquiring Transplace).
- Operational improvements, such as reduced fixed costs and increased efficiency, led to a significant reduction in net loss.
- Gains from the sale of the autonomous vehicle business and positive impacts from equity investments.
- Expansion of offerings through acquisitions (e.g., Cornershop and Drizly) added grocery and alcohol delivery capabilities.
- Growth in Monthly Active Platform Consumers and trip volumes.

Lyft:
- Widespread COVID-19 vaccine distribution and community reopenings led to a strong recovery in revenue and active riders.
- Revenue increased by 36% compared to the prior year.
- Active riders grew by 49.2% in Q4 2021 compared to Q4 2020.
- Achieved first annual Adjusted EBITDA profitability in 2021.
- Maintained a strong liquidity position ($2.3 billion in unrestricted cash, cash equivalents, and short-term investments at year-end).
- These factors supported Lyft’s ongoing recovery and expansion efforts.

In summary, both companies benefited from pandemic recovery, increased demand, and operational improvements, with Uber also seeing strong growth in its Delivery and Freight segments, and Lyft achieving profitability and maintaining strong liquidity.

In [47]:
response = await agent.run(user_msg="What are the investments made by Uber?")

Running step init_run
Step init_run produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced no event
Running step call_tool
Step call_tool produced event ToolCallResult
Running step aggregate_tool_results
Step aggregate_tool_results produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced event StopEvent


In [48]:
display(Markdown(f"{response}"))

In 2021, Uber made several notable investments and acquisitions, as detailed in their annual report:

**Acquisitions:**
- Transplace: Uber acquired 100% of Transplace, a major logistics and transportation management provider, to expand its Uber Freight business.
- Cornershop: Uber completed the acquisition of the remaining stake in Cornershop, a grocery delivery platform, making it a wholly owned subsidiary.
- Drizly: Uber acquired Drizly, an on-demand alcohol marketplace, to enhance its Delivery segment.

**Equity Investments and Stakes:**
- Aurora: Uber sold its autonomous vehicle unit (ATG) to Aurora Innovation, receiving Aurora stock in return. After Aurora’s public listing, Uber’s stake was valued at $3.4 billion at year-end.
- Moove: Uber invested in Moove, a Spanish vehicle fleet operator, acquiring a 30% minority interest and providing a $213 million loan, with an option to buy more shares.
- Didi: Uber’s investment in Didi became publicly traded shares after Didi’s IPO.
- Zomato: Uber’s investment in Zomato also became publicly traded after Zomato’s IPO, valued at $1.1 billion at year-end.

**Divestitures and Related Investments:**
- Yandex Joint Ventures: Uber sold its stakes in Yandex’s self-driving and delivery businesses, as well as part of its interest in MLU B.V., to Yandex.
- ATG Business: Uber divested its ATG business to Aurora, receiving equity in Aurora as consideration.

**Other Financial Developments:**
- Uber issued $1.5 billion in senior notes in 2021.
- Uber’s investments in Grab and Zomato resulted in significant unrealized gains, while its Didi investment saw a substantial unrealized loss.

These investments reflect Uber’s strategy to expand its logistics, delivery, and mobility services, while also restructuring and monetizing its stakes in global technology and mobility companies.

In [49]:
response = await agent.run(user_msg="Compare the investments made by Uber and lyft?")

Running step init_run
Step init_run produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced no event
Running step call_tool
Running step call_tool
Step call_tool produced event ToolCallResult
Running step aggregate_tool_results
Step aggregate_tool_results produced no event
Step call_tool produced event ToolCallResult
Running step aggregate_tool_results
Step aggregate_tool_results produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced event StopEvent


In [50]:
display(Markdown(f"{response}"))

Here is a comparison of the investments made by Uber and Lyft in 2021:

Uber

- Acquisitions: Uber acquired Cornershop (grocery delivery), Drizly (alcohol delivery), and Transplace (logistics), and consolidated CS-Mexico.
- Divestitures & Equity Investments: Sold its autonomous vehicle unit (ATG) to Aurora Innovation in exchange for equity, and restructured/sold interests in Yandex-related ventures.
- Technology & Partnerships: Invested in Moove (vehicle fleet operator in Spain), with a minority equity stake, a $213 million loan, and a commercial partnership. Continued capital expenditures on technology, including assets from Postmates, Transplace, Drizly, and Cornershop.
- Capital Expenditures: Issued $1.5 billion in senior notes for additional capital, and entered a reinsurance agreement for legacy auto insurance liabilities, resulting in a $1 billion cash inflow.
- Strategy: Focused on expanding core businesses, enhancing technology, and strengthening market position through acquisitions and partnerships.

Lyft

- Acquisitions & Technology: Invested heavily in R&D, completed strategic acquisitions, and expanded its network of shared bikes and scooters. Developed third-party self-driving technology through Lyft Autonomous.
- Divestitures: Sold its Level 5 self-driving vehicle division, shifting to licensing and data access agreements with autonomous vehicle companies.
- Partnerships & Innovation: Invested in sales/marketing, expanded Lyft Rentals, and supported the Express Drive vehicle leasing program. Received a $64 million non-marketable equity security as part of a licensing agreement.
- Capital Expenditures: Included real estate leases (81 locations), finance leases for Flexdrive, and other long-term commitments.
- Sustainability: Invested in environmental initiatives, aiming for 100% electric vehicles by 2030 and compliance with California’s Clean Miles Standard.
- Strategy: Focused on technological innovation, multimodal transportation, strategic partnerships, operational infrastructure, and sustainability.

Summary

- Uber’s investments were more acquisition-heavy, targeting expansion in delivery, logistics, and international markets, with significant capital raised and deployed for these purposes.
- Lyft’s investments focused on technology, multimodal transportation (bikes, scooters, rentals), partnerships, and sustainability, with fewer large acquisitions but significant R&D and operational investments.

Both companies made strategic divestitures in autonomous vehicle technology, but Uber retained equity in Aurora, while Lyft shifted to a partnership/licensing model. Uber’s investments were broader in scope and scale, while Lyft’s were more focused on platform innovation and sustainability.